In [9]:
# Esto es solo para poder debugear.
!pip install torch tqdm
import torch
import numpy as np
import tqdm

# Genera una semilla fija para que los experimentos sea repetibles.
t_cg = torch.manual_seed(1547)


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### NUMPY

In [2]:
x = np.array([[1, 2,3, 4]],dtype = np.float32)
y = np.array([2, 4,6,8], dtype = np.float32)

def forward(x):
  return x * w
def loss(y_pred, y):
  return np.mean(((y_pred - y)**2))

def grad(y_pred,y):
  return (x* 2 * ((y_pred) - y)).mean()

In [3]:
w= 0.0
lr =0.05
for e in range(10):
  y_pred = forward(x)
  loss_i = loss(y_pred,y)
  grad_i = grad(y_pred,y)
  w -= lr*grad_i
  print(loss_i, w)

print(forward(x = 5))


30.0 1.5
1.875 1.875
0.1171875 1.96875
0.0073242188 1.9921875
0.00045776367 1.998046875
2.861023e-05 1.99951171875
1.7881393e-06 1.9998779296875
1.1175871e-07 1.999969482421875
6.9849193e-09 1.9999923706054688
4.3655746e-10 1.9999980926513672
9.999990463256836


### TORCH METHOD 1

In [4]:
x = torch.tensor([1,2,3, 4],dtype = torch.float32, requires_grad = False)
y = torch.tensor([2,4,6,8], dtype = torch.float32, requires_grad = False)
w = torch.tensor(0.0,dtype = torch.float32, requires_grad = True)


def forward(x):
  return w *  x
def loss(y_pred, y):
  return ((y_pred - y)**2).mean()


In [10]:
lr =0.01
for e in tqdm.tqdm(range(50)):
  y_pred = forward(x)
  l  = loss(y_pred, y)
  l.backward()

  with torch.no_grad():
    w -= lr * w.grad
  w.grad.zero_()

  if e % 10 == 0:
    print(f"epoch : {e}")
    print(f"loss : {l.item():.3f}")
    print(f"w : {w.item():.3f}")

print(forward(x = 5).item())

100%|██████████| 50/50 [00:00<00:00, 3845.02it/s]

epoch : 0
loss : 30.000
w : 0.300
epoch : 10
loss : 1.163
w : 1.665
epoch : 20
loss : 0.045
w : 1.934
epoch : 30
loss : 0.002
w : 1.987
epoch : 40
loss : 0.000
w : 1.997
9.997042655944824


### TORCH METHOD 2

In [11]:
import torch.nn as nn
x = torch.tensor([1,2,3, 4],dtype = torch.float32, requires_grad = False)
y = torch.tensor([2,4,6,8], dtype = torch.float32, requires_grad = False)
w = torch.tensor(0.0,dtype = torch.float32, requires_grad = True)

#otra forma seria : instanciar clase que devuelve objeto MSEloss
# loss = nn.MSELoss(size_average=None, reduce=None, reduction='mean')

def forward(x):
  return x * w


In [12]:
lr =0.01
for e in tqdm.tqdm(range(50)):
  y_pred = forward(x)
  # simula la funcion de loss implementada a mano antes

  # Metodo 1: instanciando clase de error
  # l = loss(input = y_pred, target = y)

  # Metodo 2 : simulan la funcion de loss implementada a mano antes mediante una funcion de error built in de torch:
  l  = nn.functional.mse_loss(input = y_pred, target = y, size_average=None, reduce=None, reduction='mean')
  l.backward()

  with torch.no_grad():
    w -= lr * w.grad
  w.grad.zero_()

  if e % 10 == 0:
    print(f"epoch : {e}")
    print(f"loss : {l.item():.3f}")
    print(f"w : {w.item():.3f}")

print(forward(x = 5).item())

100%|██████████| 50/50 [00:00<00:00, 3126.95it/s]

epoch : 0
loss : 30.000
w : 0.300
epoch : 10
loss : 1.163
w : 1.665
epoch : 20
loss : 0.045
w : 1.934
epoch : 30
loss : 0.002
w : 1.987
epoch : 40
loss : 0.000
w : 1.997
9.997042655944824


### TORCH METHOD 3

In [8]:
import torch.nn as nn

x = torch.tensor([1,2,3, 4],dtype = torch.float32, requires_grad = False)
y = torch.tensor([2,4,6,8], dtype = torch.float32, requires_grad = False)
w = torch.tensor(0.0,dtype = torch.float32, requires_grad = True)

# loss class
loss = nn.MSELoss(size_average=None, reduce=None, reduction='mean')

# crea optimizer para prescendir d actualizacion de pesos a mano
optimizer = torch.optim.SGD(params = [w], lr=0.01)

def forward(x):
  return x * w


c:\Users\Koke\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
for epoch in tqdm.tqdm(range(50)):
  # forward
  y_pred = forward(x)

  # compute loss for all the batch
  l   = loss(input = y_pred, target = y)

  # compute the gradients for each w_i : dloss/dw_i (usa grafo computacional y almacena ,en el tensor .grad asociado a cada parametro, el valor del gradiente que luego usa el algoritmo de optimizacion)
  l.backward()

  # paso del optimizer (en este caso un simple SGD) equivale a lo que haciamos a mano de w += -lr*w.grad
  optimizer.step()

  if epoch % 10 == 0:
    print(f"epoch : {epoch}")
    print(f"loss : {l.item():.3f}")
    print(f"w : {w.item():.3f}")
  optimizer.zero_grad()

print(forward(x = 5).item())

100%|██████████| 50/50 [00:00<00:00, 1626.00it/s]

epoch : 0
loss : 0.000
w : 1.999
epoch : 10
loss : 0.000
w : 1.999
epoch : 20
loss : 0.000
w : 1.999
epoch : 30
loss : 0.000
w : 1.999
epoch : 40
loss : 0.000
w : 1.999
9.997042655944824


### TORCH METHOD 4

In [14]:
import torch.nn as nn

x = torch.tensor([[1],[2],[3], [4]],dtype = torch.float32, requires_grad = False)
y = torch.tensor([[2],[4],[6],[8]], dtype = torch.float32, requires_grad = False)

# Dimension layers (in and out, entra 4 samples (batch) sale 1 output por cada input) teniendo en cuenta el batch: X_input(1,4) * W(4,1)--> (1,1) // sin batch : X_input(1) * W(1)--> (1)
batch_size , features  = x.shape
# features == dim_input y en este caso features ==  dim_output pq capa solo 1 parametro (como que solo una neurona)

# model [capa lineal inicializa un tensor paramters asociado a la capa en funcion de las input y output dimesnions especificadas ]: capa lineal con 1 W y 1 X input (escalares) ; luego dim_input = 1 y dim_output = 1
model = nn.Linear(in_features= 1, out_features = 1)

# loss class
loss = nn.MSELoss(size_average=None, reduce=None, reduction='mean')

# crea optimizer para prescendir de actualizacion de pesos a mano
optimizer = torch.optim.SGD(params = model.parameters(), lr=0.01)

def forward(x):
  return x * w


In [15]:
for epoch in tqdm.tqdm(range(50)):
  # forward now is == to calling the model
  y_pred = model(x)

  # compute loss for all the batch
  l   = loss(input = y_pred, target = y)

  # compute the gradients for each w_i : dloss/dw_i (usa grafo computacional y almacena ,en el tensor .grad asociado a cada parametro, el valor del gradiente que luego usa el algoritmo de optimizacion)
  l.backward()

  # paso del optimizer (en este caso un simple SGD) equivale a lo que haciamos a mano de w += -lr*w.grad
  optimizer.step()

  if epoch % 10 == 0:
    print(f"epoch : {epoch}")
    print(f"loss : {l.item():.3f}")
    print(f"w : {w.item():.3f}")
  optimizer.zero_grad()

print(forward(x = 5).item())

100%|██████████| 50/50 [00:00<00:00, 1190.32it/s]

epoch : 0
loss : 45.416
w : 1.999
epoch : 10
loss : 1.182
w : 1.999
epoch : 20
loss : 0.037
w : 1.999
epoch : 30
loss : 0.007
w : 1.999
epoch : 40
loss : 0.006
w : 1.999
9.997042655944824


### TORCH METHOD 5  : custom module

tensor([0.2530], requires_grad=True)


In [5]:
x = torch.randn(4)
print(x.shape)
print(x)
print(x.unsqueeze(0).shape)
print(x.unsqueeze(0))


#x_soft_max = x.softmax(dim = -1)

torch.Size([4])
tensor([-0.4052, -2.1428, -0.3570, -0.1193])
torch.Size([1, 4])
tensor([[-0.4052, -2.1428, -0.3570, -0.1193]])


In [6]:
x = torch.randn(2,2)
y = torch.randn(2,2)

print(x.shape)
print(x)
x_soft_max = x.softmax(dim = -1)


print(x_soft_max.shape)
print(x_soft_max.sum(dim = 1))
print(x_soft_max)



torch.Size([2, 2])
tensor([[-1.3433,  0.4192],
        [ 0.3304,  0.9006]])
torch.Size([2, 2])
tensor([1., 1.])
tensor([[0.1465, 0.8535],
        [0.3612, 0.6388]])


In [10]:
x = torch.randn(2,2)
y = torch.randn(2,2)
print(x)
print(y)

print(x * y )




tensor([[ 0.2425, -0.6660],
        [ 0.4344, -0.5432]])
tensor([[ 0.0457,  1.4203],
        [ 0.3156, -1.1674]])
tensor([[ 0.0111, -0.9460],
        [ 0.1371,  0.6342]])


In [11]:
print(x  @ y )

tensor([[-0.1991,  1.1219],
        [-0.1516,  1.2511]])


In [86]:
suma = 0
for e in x_soft_max[:,0]:
  print(e)
  suma += e
print(suma)

tensor(0.2999)
tensor(0.7001)
tensor(1.0000)


In [87]:
suma = 0
for e in x_soft_max[1,:]:
  print(e)
  suma += e
print(suma)

tensor(0.7001)
tensor(0.2779)
tensor(0.9779)


In [82]:
a = torch.randn(1, 2, 3, 4)
a.size()
print(a)
b = a.transpose(1, 2)  # Swaps 2nd and 3rd dimension
b.size()
print(b)
c = a.view(1, 3, 2, 4)  # Does not change tensor layout in memory
c.size()
torch.equal(b, c)

tensor([[[[ 0.5207, -1.2700,  0.9622, -1.4488],
          [-0.3503, -1.8362, -0.2652,  1.5133],
          [-0.5715, -1.1178, -0.0886, -0.9020]],

         [[-0.2685,  1.1808, -0.2541,  0.6832],
          [ 0.6094, -0.1029, -0.8680,  0.7459],
          [ 0.1279, -0.4754,  0.0697, -1.5372]]]])
tensor([[[[ 0.5207, -1.2700,  0.9622, -1.4488],
          [-0.2685,  1.1808, -0.2541,  0.6832]],

         [[-0.3503, -1.8362, -0.2652,  1.5133],
          [ 0.6094, -0.1029, -0.8680,  0.7459]],

         [[-0.5715, -1.1178, -0.0886, -0.9020],
          [ 0.1279, -0.4754,  0.0697, -1.5372]]]])


False

In [ ]:
# Red estilo pytorch
class FFNN(torch.nn.Module):
  def __init__(self, d0=300, d1=200, d2=300, d3 = 3):
    super(FFNN, self).__init__()

    # Definimos capas (automáticamente se registran como parametros)
    self.fc1 = torch.nn.Linear(d0, d1, bias=True)
    self.fc2 = torch.nn.Linear(d1, d2, bias=True)
    self.fc3 = torch.nn.Linear(d2,  d3, bias=True)

  # Computa la pasada hacia adelante
  def forward(self, x):

    u1 = self.fc1(x)
    h1 = torch.tanh(u1)
    u2 = self.fc2(h1)
    h2 = torch.sigmoid(u2)
    u3 = self.fc3(h2)
    _sft_max_last = torch.nn.Softmax(dim = 1)
    y_pred = _sft_max_last(u3)

    return y_pred

In [ ]:
from torch.utils.data import Dataset, DataLoader

class RandomDataSet(Dataset):
  def __init__(self, N, f, k):
    R_N_f = torch.rand(N,f)
    self.X = torch.bernoulli(R_N_f)
    R_N_1 = torch.rand(N,k)
    #self.Y = torch.bernoulli(R_N_1)
    _sftmax = torch.nn.Softmax(dim=1)
    self.Y  = _sftmax(R_N_1)
    self.num_features = f

  # Debemos definir __len__ para retornar el tamaño del dataset
  def __len__(self):
    return self.X.size()[0]

  # Debemos definir __getitem__ para retornar el i-ésimo
  # ejemplo en nuestro dataset.
  def __getitem__(self, i):
    return self.X[i], self.Y[i]

In [ ]:

def loop_FFNN(dataset, batch_size, d1, d2, d3, lr,
                 epochs, run_in_GPU=True, reports_every=1,
                 cheq_grad=False):

  # Define un tipo para los tensores según si correrá en la GPU o no
  device = 'cuda' if run_in_GPU else 'cpu'

  # d0 es la cantidad de features
  d0 = dataset.num_features

  # Crea la red
  red = FFNN(d0, d1, d2, d3)

  # Pasa la red al dispositivo elegido
  red.to(device)

  # Muestra la cantidad de parámetros
  print('Red:', red)

  # Crea un dataloader desde el dataset
  data = DataLoader(dataset, batch_size, shuffle=True)

  # Crea un optimizador para el descenso de gradiente
  optimizador = torch.optim.SGD(red.parameters(), lr)

  # Define una perdida
  #perdida = torch.nn.BCELoss()
  perdida = torch.nn.CrossEntropyLoss()

  # Comienza el entrenamiento
  tiempo_epochs = 0
  for e in range(1,epochs+1):
    inicio_epoch = time.time()

    for (x,y) in data:
      # Asegura de pasarlos a la GPU si fuera necesario
      x, y = x.to(device), y.to(device)

      # Computa la pasada hacia adelante (forward)
      y_pred = red.forward(x)

      # Computa la función de pérdida
      L = perdida(y_pred,y)

      # Computa los gradientes hacia atrás (backpropagation)
      L.backward()

      # Descenso de gradiente para actualizar los parámetros
      optimizador.step()

      # Limpia los gradientes
      optimizador.zero_grad()

    tiempo_epochs += time.time() - inicio_epoch

    if e % reports_every == 0:
      # Calcula la certeza de las predicciones sobre todo el conjunto
      X = dataset.X.to(device)
      Y = dataset.Y.to(device)

      # Predice usando la red
      Y_PRED = red.forward(X)

      # Calcula la pérdida de todo el conjunto
      L_total = perdida(Y_PRED, Y)

      # Elige una clase dependiendo del valor de Y_PRED
      Y_PRED_BIN = (Y_PRED >= 0.5).float()

      correctos = torch.sum(Y_PRED_BIN == Y).item()
      acc = (correctos / N) * 100

      sys.stdout.write(
            '\rEpoch:{0:03d}'.format(e) + ' Acc:{0:.2f}%'.format(acc)
            + ' Loss:{0:.4f}'.format(L_total)
            + ' Tiempo/epoch:{0:.3f}s'.format(tiempo_epochs/e))

In [ ]:
N = 50000 # numero de ejemplos
f = 300 # numero de features
k = 3 # numero de clases

dataset = RandomDataSet(N,f, k)

In [ ]:
epochs = 50
loop_FFNN(dataset, batch_size=100, d1=500, d2=100, d3 = 3, epochs=epochs,
             run_in_GPU=True, lr=0.08)

Red: FFNN(
  (fc1): Linear(in_features=300, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=5, bias=True)
)


RuntimeError: 0D or 1D target tensor expected, multi-target not supported